In [ ]:
!pip install -q kagglehub


In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import kagglehub


In [ ]:
base_path = kagglehub.dataset_download(
    "manjilkarki/deepfake-and-real-images"
)

print("Dataset path:", base_path)
print("Base folder contents:", os.listdir(base_path))


Using Colab cache for faster access to the 'deepfake-and-real-images' dataset.
Dataset path: /kaggle/input/deepfake-and-real-images
Base folder contents: ['Dataset']


In [ ]:
import os

base_path = "/kaggle/input/deepfake-and-real-images"
dataset_path = os.path.join(base_path, "Dataset")  # <-- add Dataset here

train_dir = os.path.join(dataset_path, "Train")
val_dir   = os.path.join(dataset_path, "Validation")
test_dir  = os.path.join(dataset_path, "Test")

print("Train folder:", os.listdir(train_dir))
print("Validation folder:", os.listdir(val_dir))
print("Test folder:", os.listdir(test_dir))


Train folder: ['Fake', 'Real']
Validation folder: ['Fake', 'Real']
Test folder: ['Fake', 'Real']


In [ ]:
# Image settings
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load training data
train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=True
)

# Load validation data
val_gen = val_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

# ✅ Load test data
test_gen = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)



Found 140002 images belonging to 2 classes.
Found 39428 images belonging to 2 classes.
Found 10905 images belonging to 2 classes.


In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Load pre-trained MobileNetV2 base
base_model = MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

# Freeze the base layers
base_model.trainable = False

# Add custom layers on top
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)  # Binary classification

# Create final model
model = Model(inputs=base_model.input, outputs=output)


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [ ]:
# Compile the model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)


In [ ]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,422,081 (9.24 MB)

 Trainable params: 164,097 (641.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# =====================================
# Early Stopping Callback
# =====================================
early_stopping = EarlyStopping(
    monitor="val_loss",          # Monitor validation loss
    patience=3,                  # Stop after 3 epochs without improvement
    restore_best_weights=True,   # Restore best model weights
    verbose=1
)

# =====================================
# Model Checkpoint (Optional but recommended)
# Saves best model automatically
# =====================================
checkpoint = ModelCheckpoint(
    filepath="final_model/model/best_model.h5",
    monitor="val_loss",
    save_best_only=True,
    save_weights_only=False,
    verbose=1
)

# =====================================
# Train the Model
# =====================================
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=5,                   # Can safely increase due to early stopping
    callbacks=[early_stopping, checkpoint]
)

print("Training completed.")


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
4376/4376 ━━━━━━━━━━━━━━━━━━━━ 0s 497ms/step - accuracy: 0.7634 - loss: 0.4757
Epoch 1: val_loss improved from inf to 0.44564, saving model to final_model/model/best_model.h5


4376/4376 ━━━━━━━━━━━━━━━━━━━━ 2420s 549ms/step - accuracy: 0.7634 - loss: 0.4757 - val_accuracy: 0.7879 - val_loss: 0.4456
Epoch 2/5
4376/4376 ━━━━━━━━━━━━━━━━━━━━ 0s 382ms/step - accuracy: 0.8005 - loss: 0.4173
Epoch 2: val_loss did not improve from 0.44564
4376/4376 ━━━━━━━━━━━━━━━━━━━━ 1749s 400ms/step - accuracy: 0.8005 - loss: 0.4173 - val_accuracy: 0.7777 - val_loss: 0.4607
Epoch 3/5
4376/4376 ━━━━━━━━━━━━━━━━━━━━ 0s 384ms/step - accuracy: 0.8088 - loss: 0.4017
Epoch 3: val_loss improved from 0.44564 to 0.43251, saving model to final_model/model/best_model.h5


4376/4376 ━━━━━━━━━━━━━━━━━━━━ 1766s 404ms/step - accuracy: 0.8088 - loss: 0.4017 - val_accuracy: 0.7945 - val_loss: 0.4325
Epoch 4/5
4376/4376 ━━━━━━━━━━━━━━━━━━━━ 0s 386ms/step - accuracy: 0.8192 - loss: 0.3893
Epoch 4: val_loss did not improve from 0.43251
4376/4376 ━━━━━━━━━━━━━━━━━━━━ 1769s 404ms/step - accuracy: 0.8192 - loss: 0.3893 - val_accuracy: 0.7859 - val_loss: 0.4452
Epoch 5/5
4376/4376 ━━━━━━━━━━━━━━━━━━━━ 0s 382ms/step - accuracy: 0.8207 - loss: 0.3833
Epoch 5: val_loss improved from 0.43251 to 0.42754, saving model to final_model/model/best_model.h5


4376/4376 ━━━━━━━━━━━━━━━━━━━━ 1750s 400ms/step - accuracy: 0.8207 - loss: 0.3833 - val_accuracy: 0.8003 - val_loss: 0.4275
Restoring model weights from the end of the best epoch: 5.
Training completed.


In [ ]:
import os
import pickle
import json

# ===============================
# Base directory
# ===============================
BASE_DIR = "final_model"

MODEL_DIR = os.path.join(BASE_DIR, "model")
CONFIG_DIR = os.path.join(BASE_DIR, "config")

# ===============================
# Create folders
# ===============================
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(CONFIG_DIR, exist_ok=True)

# ===============================
# Save trained model
# ===============================
model_path = os.path.join(MODEL_DIR, "model.pkl")

with open(model_path, "wb") as f:
    pickle.dump(model, f)

print(f"Model saved at: {model_path}")

# ===============================
# Save config / metadata
# ===============================
model_config = {
    "model_name": "model",
    "version": "1.0",
    "framework": "tensorflow / keras"
}

with open(os.path.join(CONFIG_DIR, "model_config.json"), "w") as f:
    json.dump(model_config, f, indent=4)

print("Model saved successfully inside 'final_model/'")


Model saved at: final_model/model/model.pkl
Model saved successfully inside 'final_model/'


In [ ]:
# ===============================
# COMPLETE FINAL MODEL EXPORT (COLAB)
# ===============================

import os
import json
import shutil
from google.colab import files

# ===============================
# Base directories
# ===============================
BASE_DIR = "final_model"
MODEL_DIR = os.path.join(BASE_DIR, "model")
CONFIG_DIR = os.path.join(BASE_DIR, "config")

# ===============================
# Create folders
# ===============================
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(CONFIG_DIR, exist_ok=True)

# ===============================
# Save Keras / TensorFlow model
# (BEST PRACTICE – DO NOT USE PICKLE)
# ===============================
model.save(os.path.join(MODEL_DIR, "model.keras"))

# ===============================
# Save config / metadata
# ===============================
model_config = {
    "model_name": "model",
    "version": "1.0",
    "framework": "tensorflow / keras"
}

with open(os.path.join(CONFIG_DIR, "model_config.json"), "w") as f:
    json.dump(model_config, f, indent=4)

# ===============================
# Zip the final_model folder
# ===============================
shutil.make_archive(
    base_name="final_model",
    format="zip",
    root_dir=BASE_DIR
)

# ===============================
# Download to local system
# ===============================
files.download("final_model.zip")

print("✅ final_model saved, zipped, and downloaded successfully")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ final_model saved, zipped, and downloaded successfully
